In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3, InceptionResNetV2, DenseNet201
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
import os


2025-06-28 10:57:32.275184: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751108252.468254      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751108252.518098      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
img_height, img_width = 224, 224
batch_size = 32

data_dir = '/kaggle/input/skin-cancer-disease-ham10000/Ham10000'

# Augmentation محسن
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest',
    validation_split=0.2  # نسبة التحقق
)

train_data = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_data = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

num_classes = train_data.num_classes


Found 8015 images belonging to 7 classes.
Found 2000 images belonging to 7 classes.


In [3]:
def build_model(base_model_class, input_shape=(224, 224, 3), num_classes=7):
    base_model = base_model_class(include_top=False, weights='imagenet', input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model


In [4]:
from tensorflow.keras.callbacks import ModelCheckpoint

# =======================
# ✅ InceptionV3
# =======================
model1 = build_model(InceptionV3, num_classes=num_classes)
model1.compile(
    optimizer=Adam(learning_rate=0.00001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

checkpoint1 = ModelCheckpoint(
    'best_inceptionv3.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

model1.fit(
    train_data,
    validation_data=val_data,
    epochs=50,
    callbacks=[checkpoint1]
)

# =======================
# ✅ InceptionResNetV2
# =======================
model2 = build_model(InceptionResNetV2, num_classes=num_classes)
model2.compile(
    optimizer=Adam(learning_rate=0.00001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

checkpoint2 = ModelCheckpoint(
    'best_inceptionresnetv2.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

model2.fit(
    train_data,
    validation_data=val_data,
    epochs=50,
    callbacks=[checkpoint2]
)

# =======================
# ✅ DenseNet201
# =======================
model3 = build_model(DenseNet201, num_classes=num_classes)
model3.compile(
    optimizer=Adam(learning_rate=0.00001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

checkpoint3 = ModelCheckpoint(
    'best_densenet201.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

model3.fit(
    train_data,
    validation_data=val_data,
    epochs=50,
    callbacks=[checkpoint3]
)


I0000 00:00:1751108271.411588      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50


I0000 00:00:1751108328.596584     107 service.cc:148] XLA service 0x7bcca0002e30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1751108328.597433     107 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1751108333.865218     107 cuda_dnn.cc:529] Loaded cuDNN version 90300


  1/251 ━━━━━━━━━━━━━━━━━━━━ 5:59:31 86s/step - accuracy: 0.1875 - loss: 2.0386

I0000 00:00:1751108362.304876     107 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


251/251 ━━━━━━━━━━━━━━━━━━━━ 336s 997ms/step - accuracy: 0.5895 - loss: 1.3250 - val_accuracy: 0.6740 - val_loss: 0.9669
Epoch 2/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 176s 701ms/step - accuracy: 0.6842 - loss: 0.9342 - val_accuracy: 0.7160 - val_loss: 0.8024
Epoch 3/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 173s 689ms/step - accuracy: 0.7073 - loss: 0.8409 - val_accuracy: 0.7365 - val_loss: 0.7429
Epoch 4/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 172s 686ms/step - accuracy: 0.7334 - loss: 0.7611 - val_accuracy: 0.7615 - val_loss: 0.6800
Epoch 5/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 174s 693ms/step - accuracy: 0.7409 - loss: 0.7368 - val_accuracy: 0.7620 - val_loss: 0.6808
Epoch 6/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 174s 693ms/step - accuracy: 0.7582 - loss: 0.6830 - val_accuracy: 0.7900 - val_loss: 0.6105
Epoch 7/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 176s 699ms/step - accuracy: 0.7812 - loss: 0.6303 - val_accuracy: 0.7985 - val_loss: 0.5697
Epoch 8/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 175s 698ms/step - accuracy: 0.7841 - loss: 0.60

KeyboardInterrupt: 

In [ ]:
from tensorflow.keras.models import load_model

model1 = load_model('best_inceptionv3.h5')
model2 = load_model('best_inceptionresnetv2.h5')
model3 = load_model('best_densenet201.h5')

In [ ]:
pred1 = model1.predict(val_data)
pred2 = model2.predict(val_data)
pred3 = model3.predict(val_data)

ensemble_pred = (pred1 + pred2 + pred3) / 3

y_pred = np.argmax(ensemble_pred, axis=1)
y_true = val_data.classes
class_names = list(val_data.class_indices.keys())


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix - Ensemble')
plt.show()

print(classification_report(y_true, y_pred, target_names=class_names))


In [ ]:
import pandas as pd

df = pd.DataFrame({
    'Filename': val_data.filenames,
    'TrueLabel': y_true,
    'PredictedLabel': y_pred
})
df.to_csv('ensemble_predictions.csv', index=False)
